## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Tiksi,RU,71.6872,128.8694,39.54,90,99,7.58,overcast clouds
1,1,Qaanaaq,GL,77.4840,-69.3632,49.44,65,90,8.77,overcast clouds
2,2,Prince Rupert,CA,54.3161,-130.3201,55.49,96,100,11.50,light rain
3,3,Tahta,EG,26.7693,31.5021,86.86,25,0,10.67,clear sky
4,4,Olafsvik,IS,64.8945,-23.7142,51.57,84,100,14.76,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min = float(input('What is the minimum temperature for your vacation?'))
max = float(input('What is the maximum temperature for your vacation?'))

In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
city_data_df = city_data_df.loc[city_data_df['Max Temp'] > min]
city_data_df = city_data_df.loc[city_data_df['Max Temp'] < max]
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
13,13,San Cristobal,VE,7.7669,-72.2250,75.38,60,35,5.53,scattered clouds
21,21,Sao Filipe,CV,14.8961,-24.4956,75.06,78,100,5.66,overcast clouds
34,34,Fukuma,JP,33.7667,130.4667,75.34,94,100,5.91,overcast clouds
73,73,Ribeira Grande,PT,38.5167,-28.7000,75.60,73,20,12.66,few clouds
84,84,Matara,LK,5.9485,80.5353,76.21,92,66,8.52,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.
city_data_df.count()

City_ID                79
City                   79
Country                79
Lat                    79
Lng                    79
Max Temp               79
Humidity               79
Cloudiness             79
Wind Speed             79
Current Description    79
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_hotel_df = city_data_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_hotel_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
13,San Cristobal,VE,75.38,scattered clouds,7.7669,-72.2250,
21,Sao Filipe,CV,75.06,overcast clouds,14.8961,-24.4956,
34,Fukuma,JP,75.34,overcast clouds,33.7667,130.4667,
73,Ribeira Grande,PT,75.60,few clouds,38.5167,-28.7000,
84,Matara,LK,76.21,broken clouds,5.9485,80.5353,
85,Faanui,PF,77.65,broken clouds,-16.4833,-151.7500,
86,Puerto Ayora,EC,76.96,broken clouds,-0.7393,-90.3518,
96,Avarua,CK,75.25,broken clouds,-21.2078,-159.7750,
103,Vaini,TO,75.36,few clouds,-21.2000,-175.2000,
111,Bang Saphan,TH,76.39,overcast clouds,11.2126,99.5117,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    request = requests.get(base_url, params=params).json()

    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = request["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
    
hotel_df
    
        

Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
13,San Cristobal,VE,75.38,scattered clouds,7.7669,-72.2250,Pirineos
21,Sao Filipe,CV,75.06,overcast clouds,14.8961,-24.4956,Hotel Xaguate
34,Fukuma,JP,75.34,overcast clouds,33.7667,130.4667,Masago
73,Ribeira Grande,PT,75.60,few clouds,38.5167,-28.7000,Quinta da Meia Eira
84,Matara,LK,76.21,broken clouds,5.9485,80.5353,Amaloh Boutique Resort
...,...,...,...,...,...,...,...
640,Casablanca,MA,75.42,clear sky,33.5928,-7.6192,فندق البيت المغربي كازبلانكا
644,Buchanan,LR,75.52,light rain,5.8808,-10.0467,Kamaneahn Hotel Inc
659,Dongdu,CN,77.67,broken clouds,35.8500,117.7000,Dushi 118 Hotel
673,Honiara,SB,76.14,broken clouds,-9.4333,159.9500,Heritage Park Hotel


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df['Hotel Name']!= '']
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
13,San Cristobal,VE,75.38,scattered clouds,7.7669,-72.2250,Pirineos
21,Sao Filipe,CV,75.06,overcast clouds,14.8961,-24.4956,Hotel Xaguate
34,Fukuma,JP,75.34,overcast clouds,33.7667,130.4667,Masago
73,Ribeira Grande,PT,75.60,few clouds,38.5167,-28.7000,Quinta da Meia Eira
84,Matara,LK,76.21,broken clouds,5.9485,80.5353,Amaloh Boutique Resort
...,...,...,...,...,...,...,...
640,Casablanca,MA,75.42,clear sky,33.5928,-7.6192,فندق البيت المغربي كازبلانكا
644,Buchanan,LR,75.52,light rain,5.8808,-10.0467,Kamaneahn Hotel Inc
659,Dongdu,CN,77.67,broken clouds,35.8500,117.7000,Dushi 118 Hotel
673,Honiara,SB,76.14,broken clouds,-9.4333,159.9500,Heritage Park Hotel


In [69]:
# 8a. Create the output File (CSV)
output = "WeatherPy_Vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output, index_label="City_ID")

In [10]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [11]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)

# 11b. Display the figure

fig

Figure(layout=FigureLayout(height='420px'))